<img src ='Pic\logo.jpg' width='500px'>

# PHÂN LOẠI COMMNET (TOXIC - NON TOXIC) SỬ DỤNG HỌC MÁY (Text Classification)
---
Xử lý ngôn ngữ tự nhiên (natural language processing - NLP) là một nhánh của trí tuệ nhân tạo tập trung vào các ứng dụng trên ngôn ngữ của con người. Trong trí tuệ nhân tạo thì xử lý ngôn ngữ tự nhiên là một trong những phần khó nhất vì nó liên quan đến việc phải hiểu ý nghĩa ngôn ngữ-công cụ hoàn hảo nhất của tư duy và giao tiếp. (wikipedia)

<img src='Pic/Picture1.jpg'>

Tập dữ liệu bao gồm 56700 comment với 2 thuộc tính (class, tweet),


1.   Class cho biết comment này thuộc lớp nào:
    *   0 : Không độc - Non Toxic
    *   1 : Độc - Toxic 
2. Tweet tập hợp các commnet, dữ liệu này chưa được xử lý.


**CÁC VẤN ĐỀ GIẢI QUYẾT TRONG PROJECT:**
* Nghiên cứu các phương pháp tiền xử lý dữ liệu văn bản
* Nghiên cứu các phương pháp trích chọn đặc trưng của dữ liệu văn bản
* Nghiên cứu sử dụng các thuật toán học máy phân lớp văn bản (Toxic - Non Toxic)

<img src='Pic/Picture2.png'>

---
*Copyright: Đặng Văn Nam - FIT.HUMG  - AIAcademy @2021*


In [ ]:
import pandas as pd
import numpy as np
import nltk as nltk
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## I. TẢI TẬP DỮ LIỆU DATA_NLP VÀ QUAN SÁT TẬP DỮ LIỆU
---


In [ ]:
#Đọc file dữ liệu Data_NLP.csv vào biến data
path='Data/Data_NLP.csv'
data = pd.read_csv(path)
data.info()

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
print(data.loc[[56600],['tweet']])

In [ ]:
#Kiểm tra dữ liệu missing trong tập: --> Không có dữ liệu missing
data.isnull().sum()

## II.TIỀN XỬ LÝ DỮ LIỆU VĂN BẢN
---

Sử dụng thư viện Natural Language Tool Kit (NLTK) thực hiện xử lý các comment

NLTK là một bộ công cụ dành riêng cho Natural Language Processing và được tích hợp vào Python. Nó đang ngày càng hoàn thiện và tích hợp các công cụ mới bởi hàng nghìn lập trình viên và cộng tác viên trên khắp thế giới. NLTK bao gồm những thư viện hàm, các công cụ phân tích, các corpus, wordnet, …giúp đơn giản hóa, tiết kiệm thời gian và công sức cho các lập trình viên. Python kết hợp với NLTK là bộ công cụ hữu hiệu và mạnh mẽ nhất dành cho Natural Language Processing.

http://www.nltk.org/


In [ ]:
#Khai báo sử dụng thư viện NLTK

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [ ]:
print(nltk.__version__)

### 2.1) Làm sạch dữ liệu
----
Mục đích bước này là loại bỏ noise trong data của bạn. Đa phần noise là các thẻ HTML, JavaScript, và đương nhiên nếu cứ để noise để tiến hành xử lý sẽ dẫn đến kết quả xử lý không tốt.

Thư viện re — Regular expression operations: thao tác với biểu thức chính quy

https://regex101.com/

In [ ]:
#Thư viện re — Regular expression operations: thao tác với biểu thức chính quy
import re

#hàm decontracted thực hiện chuyển đổi các phần viết tắt thành câu đầy đủ
def decontracted(st):
    # specific
    st = re.sub(r"won\'t", "will not", st)
    st = re.sub(r"can\'t", "can not", st)
    # general
    st = re.sub(r"n\'t", " not", st)
    st = re.sub(r"\'re", " are", st)
    st = re.sub(r"\'s", " is", st)
    st = re.sub(r"\'d", " would", st)
    st = re.sub(r"\'ll", " will", st)
    st = re.sub(r"\'ve", " have", st)
    st = re.sub(r"\'m", " am", st)
    return st

In [ ]:
 #hàm clear_link thực hiện loại bỏ liên kết (link), địa chỉ email trong câu
def clear_link(st):
    #Remove links/email
    word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', 
                '', st, flags=re.MULTILINE)
    word = re.sub(r'(@[^\s]*)', "", word)
    #word = re.sub('[\W]', ' ', st)
    return word

In [ ]:
#Hàm clear_punctuation thực hiện loai bỏ các dấu câu, ký tự đặc biệt trong chuỗi
def clear_punctuation(st):
    word = re.sub(r'[^\w\s]', '',st)
    return word

In [ ]:
#Hàm clear_special loại bỏ các ký tự chỉ để lại các ký tự chữ a-z, A-Z
def clear_special(st):
    word = re.sub('[^a-zA-Z]', ' ', st)
    return word

In [ ]:
#Hàm clear_noise kết hợp sử dụng các hàm ở trên để xử lý chuỗi

def clear_noise(word):
    word = word.lower()         # chuyển toàn bộ sang chữ thường để xử lý
    word = decontracted(word)
    word = clear_link(word)
    word = clear_punctuation(word)
    word = clear_special(word)
    return word

In [ ]:
#Kiểm tra clear_noise:
text=""""SORRY JB FAN &#65292;REALLY JUSTIN COME BACK". 
That is the top comment on a Justin Timberlake video with 10 million views. 
Son of a fucking bitch! I'm looking forward this video. Have you been seen it yet. 
fanjustin@gmail.com; https://allflim.com"""
#text = """I won't give up @nam to learn English 123; Email: dangnam1985@gmail.com; web: http://dantri.com.vn"""
#print(decontracted(text))
#print(clear_link(text))
#print(clear_punctuation(text))
#print(clear_special(text))
print("Dữ liệu ban đầu: \n", text)
print('-----------------------------------------------')
text1 = clear_noise(text)
print("Dữ liệu sau khi làm sạch: \n", clear_noise(text1))

### 2.2) Loại bỏ stopword
---

StopWords là những từ xuất hiện nhiều trong ngôn ngữ tự nhiên, tuy nhiên lại không mang nhiều ý nghĩa. Ở tiếng việt StopWords là những từ như: để, này, kia... Tiếng anh là những từ như: is, that, this... Tham khảo thêm tại danh sách stopwords trong tiếng việt

Có rất nhiều cách để loại bỏ StopWords nhưng có 2 cách chính là:

* Dùng từ điển

* Dựa theo tần suất xuất hiện của từ

In [ ]:
#stopwords là những từ xuất hiện nhiều trong văn bản, nhưng ko có ý nghĩa 
#Load danh sách Stopword trong tiếng anh

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
#Hiển thị danh sách các stopwords trong tiếng anh
print('Hiển thị danh sách các stopwords trong tiếng anh:\n',stop)

In [ ]:
#hàm clear_stopwords lọai bỏ các từ stopword trong câu
def clear_stopwords(st):
    word = " ".join(st for st in st.split() if st not in stop)
    return word

In [ ]:
print('Dữ liệu ban đầu: \n',text1)
print('-------------------------------------')
text2 = clear_stopwords(text1)
print('Dữ liệu loại bỏ stopwords:\n',text2)

### 2.3) Chuẩn hóa từ (Stemming và Lemmatization)
---
Trong quá trình xử lý ngôn ngữ tự nhiên, chúng ta sẽ có nhu cầu so sánh các từ (token) với nhau. Việc so sánh này tưởng chừng như đơn giản là lấy 2 chuỗi ký tự và dùng phép “==” để kiểm tra, nhưng thực tế thì không phải là như vậy. Đối với một số ngôn ngữ, tiêu biểu là tiếng Anh, mỗi từ có thể có nhiều biến thể khác nhau. Điều này làm cho việc so sánh giữa các từ là không thể mặc dù về mặc ý nghĩa cơ bản là như nhau. Ví dụ các từ “walks“, “walking“, “walked” đều là các biến thể của từ “walk” và đều mang ý nghĩa là “đi bộ”. Vậy làm sao để so sánh các từ như thế với nhau? Lemmatization và Stemming chính là 2 kỹ thuật thường được dùng cho việc này.

Stemming là kỹ thuật dùng để biến đổi 1 từ về dạng gốc (được gọi là stem hoặc root form) bằng cách cực kỳ đơn giản là loại bỏ 1 số ký tự nằm ở cuối từ mà nó nghĩ rằng là biến thể của từ. Ví dụ như chúng ta thấy các từ như walked, walking, walks chỉ khác nhau là ở những ký tự cuối cùng, bằng cách bỏ đi các hậu tố –ed, –ing hoặc –s, chúng ta sẽ được từ nguyên gốc là walk. Người ta gọi các bộ xử lý stemming là Stemmer.

Bởi vì nguyên tắc hoạt động của stemmer rất là đơn giản như vậy cho nên tốc độ xử lý của nó rất là nhanh, và kết quả stem đôi khi không được như chúng ta mong muốn. Chẳng hạn như từ goes sẽ được stem thành từ goe (bỏ chữ s cuối từ) trong khi đó stem của từ go vẫn là go, kết quả là 2 từ “goes” và “go” sau khi được stem thì vẫn không giống nhau. Một nhược điểm khác là nếu các từ dạng bất quy tắt như went hay spoke thì stemmer sẽ không thể đưa các từ này về dạng gốc là go hay speak.

Tuy có các nhược điểm như trên nhưng trong thực tế Stemming vẫn được sử dụng khá phổ biến trong NLP vì nó có tốc độ xử lý nhanh và kết quả cuối cùng nhìn chung không hề tệ khi so với Lemmatization.

In [ ]:
from nltk.stem import WordNetLemmatizer
wn = WordNetLemmatizer()
import string
my_sw = ['rt', 'ht', 'fb', 'amp', 'gt']

def black_txt(token):
  if token == 'u':
    token = 'you'
  return  token not in stop and token not in list(string.punctuation) and token not in my_sw


def fun_stemlem(word):
  list_word_clean = []
  for w1 in word.split(" "):
    if  black_txt(w1.lower()):
      word_lemma =  wn.lemmatize(w1,  pos="v")
      list_word_clean.append(word_lemma)

  #Cleaning, lowering and remove whitespaces
  word = " ".join(list_word_clean)
  return word 

In [ ]:
print('Dữ liệu ban đầu: \n',text2)
print('-------------------------------------')
text3 = fun_stemlem(text2)
print('Dữ liệu sau khi đã chuẩn hóa:\n',text3)

In [ ]:
#Ví dụ một comment trong data:
st1 = data.iloc[2000]['tweet']
st1

In [ ]:
print('1.----------Chuỗi ban đầu------------------------:\n ', st1)
sta = clear_noise(st1)
print('2.----------Chuỗi loại bỏ nhiễu:-----------------:\n',sta)
stb = clear_stopwords(sta)
print('3.----------Chuỗi loại bỏ stopwords--------------:\n',stb)
stc = fun_stemlem(stb)
print('4.--Chuỗi loại xử lý Stemming và Lemmatization---:\n',stc)   

### 2.4) Tiền Xử lý toàn bộ tập dữ liệu với các hàm đã xây dựng
---


In [ ]:
#Xây dựng hàm prepare_data: để thực hiện tiền xử lý dữ liệu
def prepare_data(word):
    word = clear_noise(word)        #Loại bỏ nhiễu trong các comment
    word = clear_stopwords(word)    #Loại bỏ stopword trong các comment
    word = fun_stemlem(word)        #Chuẩn hóa comment
    return word
    

In [ ]:
#Hiển thị danh sách các từ trước và sau khi tiền xử lý để test
for idx in data[30090:30094].index:
  print(idx, 
        '\n a.Dữ liệu gốc          :', data.iloc[idx]['tweet'],
        '\n b.Dữ liệu sau chuẩn hóa:',prepare_data(data.iloc[idx]['tweet']))
  print("************")

In [ ]:
#Tạo dataframe data_new, bổ sung thêm field: tweet_ok là comment đã được tiền xử lý dữ liệu tương ứng
data_new = data.copy()
data_new['tweet_ok'] = data['tweet'].apply(lambda x: prepare_data(x))

In [ ]:
data_new.head(10)

### lọc và loại bỏ một số comment rỗng sau chuẩn hóa:
---
Sau khi thực hiện tiền xử lý, một số comment chỉ còn lại xâu rỗng, cần loại bỏ những xâu này


In [ ]:
data_new.info()

In [ ]:
#Lọc những comment sau khi xử lý chỉ còn là khoảng trắng
print('Tổng số:',data_new['tweet_ok'].loc[data_new.tweet_ok==r''].count())
data_new.loc[data_new.tweet_ok==r'']

In [ ]:
#Chỉ lấy các bản ghi có dữ liệu:
data_ok = data_new.loc[data_new.tweet_ok!=r'']
data_ok.info()

### 2.5 Kiểm tra mức độ cân bằng của tập dữ liệu:
---

In [ ]:
#Kiểm tra mức độ cân bằng của tập dữ liệu
print('Thông kê tập dữ liệu:')
print(data_ok.count())
print('-----------------------------------')
print('Thông kê số lượng comment theo lớp:')
x = data_ok['class'].value_counts()
print(x)

In [ ]:
#Trực quan hóa số liệu

import matplotlib.pyplot as plt
label = ['0: Non Toxic', '1: Toxic']

fig, ax = plt.subplots(1,2, figsize=(12,4))
plt.suptitle('THÔNG KÊ SỐ LƯỢNG COMMENT ĐỘC (1) - KHÔNG ĐỘC (0)')

ax[0].pie(x, labels=label, autopct='%.2f%%')
ax[1].bar(label,x, color='brown')
plt.grid(True)
plt.show()

### 2.6) Lưu kết quả đã xử lý ra file
---

Lưu dữ liệu đã xử lý ra file:
* File: data_all.csv (class - tweet - tweet_ok)
* File: data_finish.csv (class - tweet)

In [ ]:
data_ok.head()

In [ ]:
#Lưu kết quả sau khi đã xử lý ra file .csv
#File Data_all.csv Lưu trữ 3 thuộc tính: class - tweet (gốc) - tweet_ok (đã xử lý)
data_ok.to_csv('Data/Data_all.csv',index=None, header=True)

#File Data_finish.csv chỉ lưu trữ 2 thuộc tính: class - tweet_ok (đã xử lý)
data_ok[['class','tweet_ok']].to_csv('Data/Data_finish.csv',index=None, header=True)

### 2.7) Phân tách Biến độc - Biến phụ thuộc
---
* Biến độc lập X: Thuộc tính tweet_ok (nội dung comment sau khi đã được xử lý)
* Biến phụ thuộc y: class (cho biết comment đó là Toxic - Non toxic)

In [ ]:
#Tải file dữ liệu sau khi đã tiền xử lý
import pandas as pd
path='Data/data_finish.csv'
data_finish = pd.read_csv(path)
data_finish.head(10)

In [ ]:
#Biến độc lập X:
X = data_finish.loc[:,'tweet_ok']
X

In [ ]:
#Biến phụ thuộc y:
y = data_finish.loc[:,'class']
y

### 2.8) Tách dữ liệu thành tập Train - Test
---
* Tập Train (80%) sử dụng để huấn luyện model
* Tập Test (20%) sử dụng để kiểm thử độ chính xác của modoe


In [ ]:
from sklearn.model_selection import train_test_split
#Tách tập dữ liệu thành Train - Test (tỷ lệ: 0.8 - 0.2)
X_train,X_test,y_train,y_test = train_test_split(X,y,
                                                 test_size=0.2,
                                                 random_state=42)
print('1.Tập ban đầu:',data_finish.shape)
print('-------------------------------')
print('a.Tập Train: ', X_train.shape)
print('b.Tập Test: ', y_test.shape)

## III) Embedding TẬP DỮ LIỆU ĐÃ XỬ LÝ
---
Khi huấn luyện mô hình máy học để xử lý ngôn ngữ tự nhiên, thì chắc chắn dữ liệu bạn có ở dạng chữ viết, sự thật là bạn không thể đem trực tiếp dữ liệu chữ viết thô vào để huấn luyện mô hình máy học ngay được, bởi vì các mô hình máy học chỉ làm việc được trên những con số, hay chính xác hơn là tính toán trên các ma trận, véc-tơ số.

Điều này dẫn đến việc bạn phải nghĩ cách làm thế nào đó để chuyển dữ liệu chữ viết thô thành dữ liệu số thực, sau đó mới có thể đưa dữ liệu số thực này vào các mô hình học được, có rất nhiều kĩ thuật để làm việc này và gọi chung chúng là kĩ thuật Embedding.

Đó chỉ mới là ý tưởng của kĩ thuật Embedding, còn việc chuyển từ không gian này sang một không gian vec-tơ khác bạn không thể làm tùy tiện được mà phải đảm bảo:

* Không gian véc-tơ mới (hay véc-tơ số thực) phải thể hiện được bản chất của tập dữ liệu ban đầu (dữ liệu chữ viết).
* Cực tiểu hóa lượng mất mát thông tin xảy ra khi bạn chuyển sang không gian mới.

Một số kĩ thuật Embedding được sử dụng phổ biến như mạng Neural Network, PCA (Principal Component Analysis) gọi là kĩ thuật phân tích thành phần chính, TF-IDF, Bag of Word, Encoder-Decoder sử dụng trong RNN (Recurrent Neural Network) hoặc LSTM (Long-Short Term Memory), ...v.v.

Một số thư viện sử dụng kĩ thuật Embedding như sklearn, Word2vec, FastText, ...v.v.

### 3.1) TF-IDF
----
Viết tắt của thuật ngữ tiếng Anh term frequency – inverse document frequency,tf-idf là trọng số của một từ trong văn bản thu được qua thống kê thể hiện mức độ quan trọng của từ này trong một văn bản, mà bản thân văn bản đang xét nằm trong một tập hợp các văn bản.

**Các tính trọng số TF-IDF:**
* Tf- term frequency : dùng để ước lượng tần xuất xuất hiện của từ trong văn bản. Tuy nhiên với mỗi văn bản thì có độ dài khác nhau, vì thế số lần xuất hiện của từ có thể nhiều hơn . Vì vậy số lần xuất hiện của từ sẽ được chia độ dài của văn bản (tổng số từ trong văn bản đó)

    * TF(t, d) = (số lần từ t xuất hiện trong văn bản d) / (tổng số từ trong văn bản d)

* IDF- Inverse Document Frequency: dùng để ước lượng mức độ quan trọng của từ đó như thế nào . Khi tính tần số xuất hiện tf thì các từ đều được coi là quan trọng như nhau. Tuy nhiên có một số từ thường được được sử dụng nhiều nhưng không quan trọng để thể hiện ý nghĩa của đoạn văn. Vì vậy ta cần giảm đi mức độ quan trọng của những từ đó bằng cách sử dụng IDF :

    * IDF(t, D) = log_e( Tổng số văn bản trong tập mẫu D/ Số văn bản có chứa từ t )


In [ ]:
# Tính TF-IDF cho tập dữ liệu
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.feature_extraction.text as text

#Convert a collection of raw documents to a matrix of TF-IDF features.

vector = TfidfVectorizer(analyzer='word', max_features=15000, stop_words = 'english')

vector.fit(data_finish['tweet_ok'])
X_train_tfidf = vector.transform(X_train)
X_test_tfidf = vector.transform(X_test)

In [ ]:
print('1.Train:',X_train_tfidf.shape)
print('2.Test:',X_test_tfidf.shape)

In [ ]:
#(train_x[1])
print(X_train.iloc[10])
print(X_train_tfidf[10].data)
print(X_train_tfidf[10].data.shape)

## IV. MACHINE LEARNING CHO PHÂN LỚP VĂN BẢN
---


### Thuật toán Naive Bayes
---
* Naive Bayes Classifiers (NBC) thường được sử dụng trong các bài toán Text Classification
* NBC có thời gian training và test rất nhanh. Điều này có được là do giả sử về tính độc lập giữa các thành phần, nếu biết class.

* Nếu giả sử về tính độc lập được thoả mãn (dựa vào bản chất của dữ liệu), NBC được cho là cho kết quả tốt hơn so với SVM và logistic regression khi có ít dữ liệu training.

* NBC có thể hoạt động với các feature vector mà một phần là liên tục (sử dụng Gaussian Naive Bayes), phần còn lại ở dạng rời rạc (sử dụng Multinomial hoặc Bernoulli).

* Khi sử dụng Multinomial Naive Bayes, Laplace smoothing thường được sử dụng để tránh trường hợp 1 thành phần trong test data chưa xuất hiện ở training data.


#### Sử dụng MultinomialNB


In [ ]:
#Sử dụng mô hình Naive Bayes với TF-IDF
from sklearn import naive_bayes as nb

MultiNB = nb.MultinomialNB(alpha=0.75)

#huấn luyện mô hình với tập huấn luyện Train
MultiNB.fit(X_train_tfidf,y_train)

#Đánh giá độ chính xác của mô hình trên tập huấn luyện
acc_MultiNB = round(MultiNB.score(X_train_tfidf, y_train) * 100, 2)

print('Độ chính xác của mô hình trên tập huấn luyện: ', acc_MultiNB, '%')
print(MultiNB)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = MultiNB.predict(X_test_tfidf)

acc1 = round(accuracy_score(y_test, y_pred)*100, 2)
print('1.Độ chính xác của mô hình trên tập Test: ', acc1, '%')

acc2 = accuracy_score(y_test, y_pred, normalize=False)
print('2.Tổng số mẫu dự đoán đúng:', acc2, ' /', len(y_test))

In [ ]:
#Trực quan hóa ma trận Confusion matrix:

import matplotlib.pyplot as plt
import itertools
import numpy as np
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1, keepdims = True)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Ground Truth label')
    plt.xlabel('Predicted label')

In [ ]:
#Sử dụng ma trận confussion matrix kiểm tra kết quả:
from sklearn.metrics import confusion_matrix
cnf_matrix_NLP = confusion_matrix(y_test,y_pred,)
class_names = [0,1]
plot_confusion_matrix(cnf_matrix_NLP,
                      classes=class_names,
                      title='Confusion matrix - NLP')

### Ví dụ với một comment mới --> Sử dụng model đã huấn luyện để cho biết comment này là độc (1) hay không độc (0)
---

In [ ]:
dict_comment= {'Comment':['you are very handsome, i love you!', 
                         'You look like a dog, i hate you',
                         'i see a rose, do you like roses?',
                         'she is very ugly',
                         ' ']}
df_comment = pd.DataFrame(dict_comment)
df_comment

In [ ]:
df_comment['Comment_ok']  = df_comment['Comment'].apply(lambda x: prepare_data(x))
df_comment

In [ ]:
X_comment = df_comment.iloc[:,1]
X_comment

In [ ]:
X_comment_tfidf = vector.transform(X_comment)
X_comment_tfidf

In [ ]:
y_pred_new = MultiNB.predict(X_comment_tfidf)
y_pred_new